In [ ]:
import requests
import re
import pandas as pd
import numpy as np

In [ ]:
#reading my api key file
with open('api.txt') as file:
    api_key = file.read().strip()
query = "\"Fat and Oils\""

#creating a list from 5 pages of data from my API call
fats = []
for page in range(1,6):
    url = f'https://api.nal.usda.gov/fdc/v1/foods/list?api_key={api_key}&query={query}&pageNumber={page}'
    response = requests.get(url)
    fat_rows = response.json()
    fats.extend(fat_rows)

fatsdf = pd.DataFrame(fats)

In [ ]:
#sorting the data in alphabetical order
fatsdf = fatsdf.sort_values('description')

#filtering out unwanted data Types
fatsdf = fatsdf[fatsdf['dataType']=='SR Legacy']

#dropping unwanted columns
fatsdf = fatsdf.drop(columns=[ 'publicationDate', 'brandOwner', 'gtinUpc'])

#resetting the index
fatsdf.reset_index()

In [ ]:
#this function will parse through the list of dictionaries within each row of the 'foodNutrients' column
#it will then return the amount of the target nutrient along with it's name
def extract_nutrient(nutrient_list, target):
    if nutrient_list is None:
        return None
    if len(nutrient_list) == 0:
        return None
    for nutrient in nutrient_list:
        if nutrient.get("name") == target:
            return nutrient.get("amount")
    return None

#applying this function to create rows of my desired nutrient data
fatsdf["energy(Kcal)"] = fatsdf["foodNutrients"].apply(lambda items: extract_nutrient(items, "Energy"))
fatsdf["protein(g)"] = fatsdf["foodNutrients"].apply(lambda items: extract_nutrient(items, "Protein"))
fatsdf["saturated_fat(g)"] = fatsdf["foodNutrients"].apply(lambda items: extract_nutrient(items, "Fatty acids, total saturated"))
fatsdf["trans_fat(g)"] = fatsdf["foodNutrients"].apply(lambda items: extract_nutrient(items, "Fatty acids, total trans"))
fatsdf["monounsat_fat(g)"] = fatsdf["foodNutrients"].apply(lambda items: extract_nutrient(items, "Fatty acids, total monounsaturated"))
fatsdf["polyunsat_fat(g)"] = fatsdf["foodNutrients"].apply(lambda items: extract_nutrient(items, "Fatty acids, total polyunsaturated"))
fatsdf["cholesterol(mg)"] = fatsdf["foodNutrients"].apply(lambda items: extract_nutrient(items, "Cholesterol"))
fatsdf["sodium(mg)"] = fatsdf["foodNutrients"].apply(lambda items: extract_nutrient(items, "Sodium, Na"))

In [ ]:
#handling all NaNs and imputing them with 0.0 for cleanliness
fatsdf = fatsdf.fillna(0.0)

#renaming the description column
fatsdffinal = fatsdf.rename(columns={'description': 'Fat/Oil Name'})

#dropping final irrelevant columns
fatsdffinal = fatsdffinal.drop(columns=['fdcId', 'foodNutrients', 'dataType', 'ndbNumber'])

In [ ]:
#exporting my df to a csv
fatsdffinal.to_csv('fatandoils.csv')

links I used to make this happen: 
- https://fdc.nal.usda.gov/api-guide
- https://fdc.nal.usda.gov/food-search?type=SR%20Legacy&SRFoodCategory=Fats%20and%20Oils
- https://app.swaggerhub.com/apis/fdcnal/food-data_central_api/1.0.1#/FDC/getFoodsList